# Divide & conquer

Divide & conquer is a method for designing algorithms that solve problems by breaking the problem into smaller subproblems and then solving those smaller problems by dividing further until some base case is reached when the problem is small enough. Once the base case is reached we solve that subproblem and then use that solution to solve the larger problem by combining it with solutions to the other smaller subproblems we have solved. Merge sort is a nice example of this that we will go over. As you can probably guess from this description divide & conquer is very closely tied to the concept of recursion.


## Scalar multiplication  

The first problem divide & conquer methods are applied to in {cite:p}`dasgupta2008algorithms` is integer multiplication. 

To multiple two complex numbers $g=a+ib$ and $h=c+id$ we compute

$$
(a+ib)(c+id)= ac-bd + (bc+ad)i
$$

which involves 4 multiplications of real numbers. Carl Gauss, the famous mathematician, discovered that we can actually reduce this to 3 since 

$$
bc+ad = (a+b)(c+d) - ac - bd
$$

our formula now becomes

$$
\begin{align*}
(a+ib)(c+id) &= ac-bd + (bc+ad)i \\
&= ac-bd + ((a+b)(c+d) - ac - bd)i.
\end{align*}
$$

It seems like this involves more multiplication but actually we now just need to compute $ac$, $bd$ and $(a+b)(c+d)$ since $ac$ and $bd$ appear twice in the expression so we have 3 *unique* multiplications of real numbers.


This might seem like minimal improvement but lets see what happens when we apply recursion and switch to integer multiplication. Suppose $y$ and $x$ are $n$-bit integers where $n$ is a power of 2. First lets start by splitting $x$ and $y$ into two halves that are $\frac{n}{2}$ bits each such that 

$$
x = 2^{\frac{n}{2}} x_L + x_R \quad \text{ and } \quad y = 2^{\frac{n}{2}} y_L + y_R.
$$

For example if  $x=10110110_2$ then $x_L=1011_2$ and $x_R=0110_2$ and $x=(2^{\frac{n}{2}} \times 1011_2) + 0110_2$, (note the subscript of 2 means the number is written in base 2 i.e. binary).

Now the product of $x$ and $y$ is given by

$$
\begin{align*}
xy &= (2^{\frac{n}{2}} x_L + x_R)(2^{\frac{n}{2}} y_L + y_R)  \\
   &= 2^nx_Ly_L + 2^{\frac{n}{2}}(x_Ly_R + x_Ry_L) + x_Ry_R
\end{align*}
$$

The addition will take linear time, in the number of bits, and so do will the power of 2 multiplications since it is just a bit shift to the left i.e `a << n` or `a << n//2` in  `Python`. The important operation are the 4 multiplications of the $\frac{n}{2}$-bit numbers. Notice that we have *divided* the problem into 4 subproblems each of which are *half* the size i.e. $\frac{n}{2}$. We can now recursively perform the same routine for the 4 new multiplications which would further divide the problem into subproblem of smaller size. 
We can describe the runtime (time complexity) of these recursive algorithms using *recurrence relations* which we cover in the next section. The recurrence relation for this algorithm is given by

$$
T(n) = 4T\left(\frac{n}{2}\right) + O(n)
$$

we multiply by 4 since we are creating 4 new subproblems and multiple by $T\left(\frac{n}{2}\right)$ since the new subproblems have size $\frac{n}{2}$. The additional $O(n)$ is there to capture the linear time complexity of additions and leftward bit shifts. Solving this recurrence relation for $T(n)$ we get the solution is $T(n)=O(n^2)$. This is the same complexity as the grade-school multiplication method so we have no real improvement with this new recursive algorithm.

We can improve this algorithm by using the insight from Gauss that we laid out earlier that. Since $x_Ly_R + x_Ry_L=(x_L+x_R)(y_L+y_R)-x_Ly_L-x_Ry_R$ we can reduce the 4 multiplications needed down to 3 resulting in a more efficient algorithm. 

The pseudocode and for the algorithm are given below.

```{prf:algorithm} Integer Multiplication
:class: dropdown
:label: gauss-int-mult 
**function** $\text{gauss_int_mult}(x,y,n)$:

**Inputs** Given $n$-bit integers $x$ and $y$

**Output** Their integer product $xy$

1. if $n=1$ 
   1. return $xy$
2. $x_L = \text{ leftmost } \lceil \frac{n}{2} \rceil \text{ bits of } x$
3. $x_R = \text{ rightmost } \lfloor \frac{n}{2} \rfloor \text{ bits of } x$
4. $y_L = \text{ leftmost } \lceil \frac{n}{2} \rceil \text{ bits of } y$
5. $y_R = \text{ rightmost } \lfloor \frac{n}{2} \rfloor \text{ bits of } y$
6. $P_1 = \text{fast_int_mult}(x_L,y_L)$ $\quad \triangleright$ Compute $x_Ly_L$
7. $P_2 = \text{fast_int_mult}(x_R,y_R)$ $\quad \triangleright$ Compute $x_Ry_R$
8. $P_3 = \text{fast_int_mult}(x_L+x_R,y_L+y_R)$ $\quad \triangleright$ Compute $(x_L+x_R)(y_L+y_R)$
9. return $2^{n} P_1 + 2^{n}(P_3 - P_1 - P_2) + P_2$

```

The time complexity for this algorithm is given by the recurrence relation

$$
T(n) = 3T\left(\frac{n}{2}\right) + O(n)
$$

which has the solution $T(n) = O(n^{1.59})$ which is a very nice improvement.

## Solving recurrence relations

A recurrence relation, which we'll call recurrence for short from now on, is an equation that describes a function in terms of it's values on other, usually smaller, arguments. To solve these recurrences there are three main methods described below {cite:p}`cormen2022introduction`.

### Substitution

Given a recurrent algorithm we guess the solution and use induction to prove it is correct.


### Recursion-tree 

Given a recurrent algorithm you draw out a the recursion as a tree with with the number of branches at each node being the number of subproblems being created, the depth being the levels of recursion and the nodes of the tree representing the cost of an operation at that level of recursion. By examining the tree you then attempt to write out the expression for the time complexity by summing up the nodes.

### Master method

Given a recurrent algorithm if we can express it's runtime as a recurrence of the form

$$
T(n) = aT\left(\left\lceil\frac{n}{b}\right\rceil\right) + O(n^d)
$$

where $n$ is the size of the initial problem, $a$ is the number of subproblems that we divide the problem into at each recursion level, $\frac{n}{b}$ is the size of the subproblems and $O(n^d)$ is the time needed to combine the solutions of said subproblems into solutions for the larger problems we can use the *master theorem* to solve the recurrence. 

```{prf:theorem} Master Theorem
:class: dropdown
Given the recurrence 

$$
T(n) = aT\left(\left\lceil\frac{n}{b}\right\rceil\right) + O(n^d)
$$

the solution $T(n)$ is given by

$$
T(n) = \begin{cases}
  O(n^d)  & \text{if } d > \log_b(a)\\
  O(n^d\log(n))  & \text{if } d = \log_b(a)\\
  O(n^{\log_b(a)})  & \text{if } d < \log_b(a)\\
\end{cases}
$$

with the proof along with the theorem given in {cite:p}`dasgupta2008algorithms`
```

## Square matrix multiplication

Let $A=(a_{ik})$ and $B=(b_{jk})$ be $n\times n$ matrices. The product $AB$ is given by (recall that this is equivalent to taking the dot product between the rows of $A$ with the columns of $B$)

$$
c_{ij} = \sum_{k=1}^{n} a_{ik}b_{kj}
$$

## Merge sort

## Fast Fourier transform (FFT)

{cite:p}`dasgupta2008algorithms` explains how one can 

In [1]:
%load_ext watermark
%watermark -n -u -v -iv

Last updated: Sun Jul 14 2024

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.22.2

